# 誤差最小化と最適化

機械学習(特に教師あり学習)の目的は、パラメーターを調整して予測値 $\hat{y}$ を正解 $y$ に近づけること(尤度最大化)や予測値 $\hat{y}$ と正解 $y$ との差を小さくすること(誤差最小化)。尤度を最大化するのが計算的に難しい場合もあり、誤差最小化の問題として解いてパラメーターを求めることが多い。この尤度最大化や誤差最小化でパラメーターを求めることを最適化という。

誤差を単純に正解と予測の差$\left(y-\hat{y}\right)$とすると、負の値を取りうるので最小化できない。正解 $y$ と予測値 $\hat{y}$ から誤差を求める方法はいくつかある。この誤差(Error)を求める関数のことをコスト関数(Cost function)・損失関数(Loss function)・目的関数(Objective function)などと呼ぶ。(厳密にはそれぞれ違うものを指している場合もある)

## 平均二乗誤差(Mean Squared Error, MSE)

回帰問題の誤差として最も使われる関数。正解 $y$ に含まれるノイズがガウス分布に従う場合に正しい選択だが、世の中の大抵のノイズはガウス分布に従うのでほとんどの場面で使える。

$\begin{eqnarray}MSE=\frac{1}{n}\sum_{i=1}^n{\left(y_i-\hat{y}_i\right)^2}\end{eqnarray}$

pythonのコードで表すと

> ```python
> # n: サンプル(学習)データの数
> # y: 正解データ(i番目のサンプルの正解がy[i])
> # y_hat: 予測値(i番目のサンプルの予測値がy_hat[i])
> mse = np.power(y - y_hat, 2).sum() / n
> ```

差を二乗しているので、正解と予測値の差が最も小さいところが最小誤差になる。

## 交差エントロピー誤差(Cross-entropy Error)

分類問題の誤差としてよく使われる関数。分類の問題を確率回帰の問題に捉え直して誤差を計算している。

$\begin{eqnarray}CrossEntropy=\sum_{i=1}^n\left[-y_ilog\left(\hat{y}_i\right)-\left(1-y_i\right)log\left(1-\hat{y}_i\right)\right]\end{eqnarray}$

pythonのコードで表すと

> ```python
> # 二値分類の場合、正解ラベルは0か1で表現
> # 予測は正解ラベル1のクラスに属する確率で表現
> # 多クラス分類問題の正解ラベルはone-hot encodingで表現
> # 例えば正解ラベルが5クラスあって、あるサンプルの正解がクラス3なら[0, 0, 1, 0, 0]を渡す
> # 予測は各クラスへの所属確率なので、それぞれ0~1の範囲の値を取り、全クラスの予測を合計すると1になる
> # 例：[0.01, 0.04, 0.9, 0.03, 0.02]
> cross_entropy = (- y * np.log(y_hat) - (1 - y) * np.log(1 - y_hat)) # 2クラス
> cross_entropy = (- y * np.log(y_hat) - (1 - y) * np.log(1 - y_hat)).sum(axis=1) # 多クラス
> ```

### 導出

高校生レベルの数学で導出できる。

入力を $x$、現在のパラメーターを $\theta$、正解ラベルを $y$、予測値を $\hat{y}=\phi\left(x,\ \theta\right)$、予測値が正解ラベルに属する確率を $p\left(y\ |\ x,\ \theta\right)$ とする。例えば、$\hat{y}=\phi\left(x,\ \theta\right)=0.9$ だとして $y=1$ のとき $p\left(y\ |\ x,\ \theta\right)=0.9$、$y=0$のとき $p\left(y\ |\ x,\ \theta\right)=0.1$。そして最大化したい尤度 $L$ を以下のように定義する。

$\begin{eqnarray}
    L&=&p\left(y_1\ |\ x_1,\ \theta_1\right)\times p\left(y_2\ |\ x_2,\ \theta_2\right)...p\left(y_n\ |\ x_n,\ \theta_n\right) \\
    &=&\prod_{i=1}^n{p\left(y_i\ |\ x_i,\ \theta_i\right)}
\end{eqnarray}$

正解ラベル $y$ は$0$か$1$しか取らないので

$p\left(y_i\ |\ x_i,\ \theta_i\right)=\begin{cases}
    \hat{y}_i & (y=1) \\
    1-\hat{y}_i & (y=0)
\end{cases}$

これを利用すると、以下のように変形できる。

$\begin{eqnarray}L=\prod_{i=1}^n{\left(\hat{y}_i\right)^{y_i}\left(1-\hat{y}_i\right)^{1-y_i}}\end{eqnarray}$

確率の積よりも、その対数尤度(log-likelihood) $l$ の和にしたほうが扱いやすい。

$\begin{eqnarray}
    l&=&logL \\
    &=&\sum_{i=1}^n\left[y_ilog\left(\hat{y}_i\right)+\left(1-y_i\right)log\left(1-\hat{y}_i\right)\right]
\end{eqnarray}$

符合を反転させると、最小化したい誤差になる

$\begin{eqnarray}CrossEntropy=\sum_{i=1}^n\left[-y_ilog\left(\hat{y}_i\right)-\left(1-y_i\right)log\left(1-\hat{y}_i\right)\right]\end{eqnarray}$

### グラフ

正解に近いと誤差が0に近くなり、遠いと誤差が無限大に近づいていく。

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

eps = 1e-8
y_hat = np.linspace(eps, 1 - eps, 200)
cost_pos = - np.log(y_hat)
cost_neg = - np.log(1 - y_hat)

plt.plot(y_hat, cost_pos, label='y = 1')
plt.plot(y_hat, cost_neg, label='y = 0')

plt.xlim(0, 1)
plt.ylim(0, 5)
plt.xlabel('prediction')
plt.ylabel('cost')
plt.legend(loc='upper center')

plt.show()

## 勾配降下法(Gradient Descent)

勾配降下法は最適化手法の1つで、コスト関数をパラメーター $\theta$ についての関数 $J(\theta)$ とみなし、$J(\theta)$ の勾配を下っていく( $J(\theta)$ の傾きと逆に進む)ことでコスト関数の最小値を目指す。(図は説明のためのもので、計算式は実際の勾配降下法とは異なる)

In [ ]:
def J(theta):
    return theta ** 2

def J_grad(theta):
    return theta * 2

def gradient_descent(theta, alpha, n_step):
    thetas = np.zeros(n_step)
    costs = np.zeros(n_step)
    grads = np.zeros((n_step, 2))

    for step in range(n_step):
        thetas[step] = theta
        costs[step] = J(theta)
        grad = alpha * J_grad(theta)
        theta -= grad
        grads[step] = [- grad, J(theta) - J(thetas[step])]

    return (thetas, costs, grads)

In [ ]:
alpha = .3 # 学習率
theta = 9 # パラメーターの初期値
n_step = 3 # パラメーターの更新回数

plt.title('Concept of gradient descent 1D')

x = np.linspace(-10, 10, 30)
y = J(x)
plt.plot(x, y)

thetas, costs, grads = gradient_descent(theta, alpha, n_step)
plt.quiver(thetas, costs, grads[:, 0], grads[:, 1], angles='xy', scale_units='xy', scale=1, color='red')

plt.xlabel('$\\theta$')
plt.ylabel('cost')
plt.xlim(x.min() - 5, x.max() + 5)
plt.ylim(y.min() - 5, y.max())
plt.xticks(())
plt.yticks(())

plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

alpha = .3
theta = np.array([9, 9])
n_step = 3

def J_2d(theta):
    return np.power(theta, 2).sum()

def J_grad_2d(theta):
    return theta * 2

def gradient_descent_2d(theta, alpha, n_step):
    thetas = np.zeros(tuple([n_step]) + theta.shape)
    costs = np.zeros(n_step)
    grads = np.zeros((n_step, len(theta) + 1))
    
    for step in range(n_step):
        thetas[step] = theta
        costs[step] = J_2d(theta)
        grad = alpha * J_grad_2d(theta)
        theta = theta - grad
        grads[step] = np.concatenate((- grad, np.array([J_2d(theta) - J_2d(thetas[step])])))
    
    return (thetas, costs, grads)

x = np.linspace(-10, 10, 30)
y = np.linspace(-10, 10, 30)

xx, yy = np.meshgrid(x, y)
grid = np.c_[xx.ravel(), yy.ravel()]

z = np.power(grid, 2).sum(axis=1).reshape(xx.shape)

x_min, x_max = x.min(), x.max()
y_min, y_max = y.min(), y.max()
z_min, z_max = z.min() - 20, z.max()

fig = plt.figure()
ax = Axes3D(fig, elev=25, azim=-75)

ax.set_title('Concept of gradient descent 2D')

ax.plot_wireframe(xx, yy, z, rstride=2, cstride=2, alpha=.3)
ax.contour(xx, yy, z, zdir='z', offset=z_min)

thetas, costs, grads = gradient_descent_2d(theta, alpha, n_step)

for qx, qy, qz, qu, qv, qw in zip(thetas[:, 0], thetas[:, 1], costs, grads[:, 0], grads[:, 1], grads[:, 2]):
    length = np.sqrt(np.power([qu, qv, qw], 2).sum())
    ax.quiver(qx, qy, qz, qu, qv, qw, pivot='tail', length=length, arrow_length_ratio=min(.3, 3.0 / length), color='red')

x_flat = np.append(thetas[:, 0], thetas[-1, 0] + grads[-1, 0])
y_flat = np.append(thetas[:, 1], thetas[-1, 1] + grads[-1, 1])
z_flat = [z_min] * (n_step + 1)
ax.plot(x_flat, y_flat, z_flat, marker='x', color='red')

ax.set_xlabel('$\\theta_1$')
ax.set_ylabel('$\\theta_2$')
ax.set_zlabel('cost')
ax.set_xlim(x_min, x_max)
ax.set_ylim(y_min, y_max)
ax.set_zlim(z_min, z_max)
ax.set_xticks(())
ax.set_yticks(())
ax.set_zticks(())

plt.show()

学習率 $\alpha$ は大きすぎるとコストが発散してしまい、小さすぎると収束に時間がかかりすぎる。

In [ ]:
def draw(ax, theta, alpha, n_step):
    ax.plot(x, y)

    thetas, costs, grads = gradient_descent(theta, alpha, n_step)
    ax.quiver(thetas, costs, grads[:, 0], grads[:, 1], angles='xy', scale_units='xy', scale=1, color='red')

    ax.set_xlim(x.min() - 5, x.max() + 5)
    ax.set_ylim(y.min() - 5, y.max())
    ax.set_xticks(())
    ax.set_yticks(())

x = np.linspace(-10, 10, 30)
y = J(x)

fig = plt.figure(figsize=(6, 3))

ax = fig.add_subplot(121)
ax.set_title('Too large')
draw(ax, 4, 1.1, 5)

ax = fig.add_subplot(122)
ax.set_title('Too small')
draw(ax, 9, .03, 10)

plt.show()